TELENGANA BUS ROUTES

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_telengana = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_telengana.to_csv('Telangana_bus_details.csv', index=False)

driver.quit()

NameError: name 'driver' is not defined

In [2]:
df_telengana

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 64022,Rajdhani (AC Semi Sleeper 2+2),19:05,04h 25m,23:30,2.9,INR 389,29 Seats available
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8677,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",19:30,04h 10m,23:40,3.9,INR 320,30 Seats available
2,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8625,RAJDHANI (A.C. Semi Sleeper),21:00,04h 00m,01:00,3.8,INR 389,35 Seats available
3,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 8619,Rajdhani (AC Semi Sleeper 2+2),22:00,04h 00m,02:00,3.9,INR 389,22 Seats available
4,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...,TSRTC - 64593,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",22:30,03h 55m,02:25,4.7,INR 320,20 Seats available
...,...,...,...,...,...,...,...,...,...,...
780,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Vandana Travels,A/C Seater / Sleeper (2+1),21:10,11h 50m,09:00,0,INR 4000,40 Seats available
781,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Vandana Travels,A/C Seater / Sleeper (2+1),21:10,11h 50m,09:00,0,INR 4000,40 Seats available
782,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Mahi Trans Solutions,A/C Sleeper (2+1),19:00,12h 15m,07:15,0,INR 4999,36 Seats available
783,Hyderabad to Tirupati,https://www.redbus.in/bus-tickets/hyderabad-to...,Orange Tours And Travels,A/C Sleeper (2+1),19:00,12h 00m,07:00,0,INR 1000,3 Seats available


ANDHRAPRADESH BUS ROUTES

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_andhrapradesh = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_andhrapradesh.to_csv('ap_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/madanapalli-to-bangalore: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF68CD69632+30946]
	(No symbol) [0x00007FF68CD1E3C9]
	(No symbol) [0x00007FF68CC16FDA]
	(No symbol) [0x00007FF68CC6822C]
	(No symbol) [0x00007FF68CC6850C]
	(No symbol) [0x00007FF68CCADCB7]
	(No symbol) [0x00007FF68CC8CAAF]
	(No symbol) [0x00007FF68CCAB041]
	(No symbol) [0x00007FF68CC8C813]
	(No symbol) [0x00007FF68CC5A6E5]
	(No symbol) [0x00007FF68CC5B021]
	GetHandleVerifier [0x00007FF68CE9F83D+1301229]
	GetHandleVerifier [0x00007FF68CEABDB7+1351783]
	GetHandleVerifier [0x00007FF68CEA2A03+1313971]
	GetHandleVerifier [0x00007FF68CD9DD06+245686]
	(No symbol) [0x00007FF68CD2758F]
	(No symbol) [0x00007FF68CD23804]
	(No symbol) [0x00007FF68CD23992]
	(No symbol) [0x00007FF68CD1A3EF]
	BaseThreadInitThunk [0x00007FFBA00B7614+20]
	RtlUserThreadStart [0x00007FFBA09E26A1+33]



NameError: name 'driver' is not defined

In [4]:
df_andhrapradesh

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 9316,Super Luxury (Non AC Seater 2+2 Push Back),20:20,07h 50m,04:10,4.0,INR 469,23 Seats available
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 35189,DOLPHIN CRUISE (VOLVO / SCANIA A.C Multi Axle),20:30,04h 35m,01:05,3.8,INR 670,15 Seats available
2,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 4046,Super Luxury (Non AC Seater 2+2 Push Back),21:00,06h 40m,03:40,4.4,INR 469,16 Seats available
3,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 9651,Super Luxury (Non AC Seater 2+2 Push Back),21:00,07h 41m,04:41,4.0,INR 469,23 Seats available
4,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...,APSRTC - 9654,Super Luxury (Non AC Seater 2+2 Push Back),21:40,07h 40m,05:20,3.0,INR 469,31 Seats available
...,...,...,...,...,...,...,...,...,...,...
2009,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,SLC Road Lines,A/C Sleeper (2+1),23:59,06h 41m,06:40,3.6,INR 1750,1 Seat available
2010,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,Morning Star Travels,NON A/C Sleeper (2+1),22:30,06h 50m,05:20,1.3,INR 590,16 Seats available
2011,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,IRA Travels,A/C Sleeper (2+1),23:55,06h 15m,06:10,4.4,INR 719,8 Seats available
2012,Anantapur (andhra pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/ananthapur-t...,S.L Travels,Non A/C Seater / Sleeper (2+1),23:55,06h 40m,06:35,4.1,INR 1000,31 Seats available


KERALA BUS ROUTES

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_kerala = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_kerala.to_csv('kerala_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/coimbatore-to-ooty: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF68CD69632+30946]
	(No symbol) [0x00007FF68CD1E3C9]
	(No symbol) [0x00007FF68CC16FDA]
	(No symbol) [0x00007FF68CC6822C]
	(No symbol) [0x00007FF68CC6850C]
	(No symbol) [0x00007FF68CCADCB7]
	(No symbol) [0x00007FF68CC8CAAF]
	(No symbol) [0x00007FF68CCAB041]
	(No symbol) [0x00007FF68CC8C813]
	(No symbol) [0x00007FF68CC5A6E5]
	(No symbol) [0x00007FF68CC5B021]
	GetHandleVerifier [0x00007FF68CE9F83D+1301229]
	GetHandleVerifier [0x00007FF68CEABDB7+1351783]
	GetHandleVerifier [0x00007FF68CEA2A03+1313971]
	GetHandleVerifier [0x00007FF68CD9DD06+245686]
	(No symbol) [0x00007FF68CD2758F]
	(No symbol) [0x00007FF68CD23804]
	(No symbol) [0x00007FF68CD23992]
	(No symbol) [0x00007FF68CD1A3EF]
	BaseThreadInitThunk [0x00007FFBA00B7614+20]
	RtlUserThreadStart [0x00007FFBA09E26A1+33]



NameError: name 'driver' is not defined

In [6]:
df_kerala

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 1586,Super Express Non AC Seater Air Bus (2+2),21:31,09h 05m,06:36,3.2,INR 640,31 Seats available
1,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 2133,Swift Deluxe Non AC Air Bus (2+2),22:12,09h 24m,07:36,3.4,INR 640,12 Seats available
2,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 2063,SWIFT-GARUDA A/C SEATER BUS,22:32,09h 03m,07:35,3.9,INR 627,7 Seats available
3,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 574,Super Express Non AC Seater Air Bus (2+2),23:00,10h 30m,09:30,3.9,INR 469,18 Seats available
4,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...,KSRTC (Kerala) - 583,Super Express Non AC Seater Air Bus (2+2),23:30,10h 05m,09:35,3.5,INR 469,15 Seats available
...,...,...,...,...,...,...,...,...,...,...
359,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Cochin Express,Volvo Multi-Axle B9R A/c Semi Sleeper (2+2),22:25,01h 34m,23:59,4.0,INR 667,13 Seats available
360,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),21:30,01h 59m,23:29,3.8,INR 750,26 Seats available
361,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),19:30,03h 59m,23:29,3.8,INR 750,26 Seats available
362,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,A1 Travels,Bharat Benz A/C Sleeper (2+1),21:00,02h 30m,23:30,3.3,INR 500,20 Seats available


UTTARPRADESH BUS ROUTES

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_uttarpradesh = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_uttarpradesh.to_csv('up_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/bareilly-to-agra: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF68CD69632+30946]
	(No symbol) [0x00007FF68CD1E3C9]
	(No symbol) [0x00007FF68CC16FDA]
	(No symbol) [0x00007FF68CC6822C]
	(No symbol) [0x00007FF68CC6850C]
	(No symbol) [0x00007FF68CCADCB7]
	(No symbol) [0x00007FF68CC8CAAF]
	(No symbol) [0x00007FF68CCAB041]
	(No symbol) [0x00007FF68CC8C813]
	(No symbol) [0x00007FF68CC5A6E5]
	(No symbol) [0x00007FF68CC5B021]
	GetHandleVerifier [0x00007FF68CE9F83D+1301229]
	GetHandleVerifier [0x00007FF68CEABDB7+1351783]
	GetHandleVerifier [0x00007FF68CEA2A03+1313971]
	GetHandleVerifier [0x00007FF68CD9DD06+245686]
	(No symbol) [0x00007FF68CD2758F]
	(No symbol) [0x00007FF68CD23804]
	(No symbol) [0x00007FF68CD23992]
	(No symbol) [0x00007FF68CD1A3EF]
	BaseThreadInitThunk [0x00007FFBA00B7614+20]
	RtlUserThreadStart [0x00007FFBA09E26A1+33]

Error occurred while scraping bus details for https://www.redbus.in/bus-ticke

NameError: name 'driver' is not defined

In [10]:
df_uttarpradesh

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - STD0354,Ordinary Non AC Seater 2+3,21:27,06h 15m,03:42,3.3,INR 448,52 Seats available
1,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0139,Janrath AC Seater 2+2,21:31,05h 59m,03:30,3.1,INR 598,36 Seats available
2,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0121,Shatabdi AC Seater 2+2,22:01,05h 00m,03:01,3.2,INR 598,37 Seats available
3,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - STD0163,Ordinary Non AC Seater 2+3,22:09,07h 16m,05:25,3.3,INR 448,52 Seats available
4,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - KSB0080,Shatabdi AC Seater 2+2,22:31,06h 15m,04:46,3.3,INR 598,36 Seats available
...,...,...,...,...,...,...,...,...,...,...
563,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,ROYAL SAFARI INDIA TRAVELS,Bharat Benz A/C Semi Sleeper (2+2),22:30,07h 30m,06:00,2.4,INR 799,31 Seats available
564,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Rjroadrunner,A/C Seater / Sleeper (2+2),22:30,06h 00m,04:30,0,INR 699,58 Seats available
565,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Sonu Nath Bus Service,AC Seater (2+2),21:00,08h 10m,05:10,0,INR 2000,49 Seats available
566,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...,Shri Bawa Lal Tour and Travels,A/C Seater/Sleeper (2+1),22:30,07h 30m,06:00,0,INR 400,28 Seats available


WESTBENGAL BUS ROUTES

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_westbengal = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_westbengal.to_csv('wb_bus_details.csv', index=False)

driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/durgapur-to-kolkata: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF68CD69632+30946]
	(No symbol) [0x00007FF68CD1E3C9]
	(No symbol) [0x00007FF68CC16FDA]
	(No symbol) [0x00007FF68CC6822C]
	(No symbol) [0x00007FF68CC6850C]
	(No symbol) [0x00007FF68CCADCB7]
	(No symbol) [0x00007FF68CC8CAAF]
	(No symbol) [0x00007FF68CCAB041]
	(No symbol) [0x00007FF68CC8C813]
	(No symbol) [0x00007FF68CC5A6E5]
	(No symbol) [0x00007FF68CC5B021]
	GetHandleVerifier [0x00007FF68CE9F83D+1301229]
	GetHandleVerifier [0x00007FF68CEABDB7+1351783]
	GetHandleVerifier [0x00007FF68CEA2A03+1313971]
	GetHandleVerifier [0x00007FF68CD9DD06+245686]
	(No symbol) [0x00007FF68CD2758F]
	(No symbol) [0x00007FF68CD23804]
	(No symbol) [0x00007FF68CD23992]
	(No symbol) [0x00007FF68CD1A3EF]
	BaseThreadInitThunk [0x00007FFBA00B7614+20]
	RtlUserThreadStart [0x00007FFBA09E26A1+33]

Error occurred while scraping bus details for https://www.redbus.in/bus-ti

NameError: name 'driver' is not defined

In [12]:
df_westbengal

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Satya Paribahan,A/C Seater (2+3),23:00,06h 10m,05:10,3.1,324,33 Seats available
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Snemita Paribahan (Shinjini),AC Seater (2+3),23:35,05h 25m,05:00,3.2,INR 420,4 Seats available
2,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Arunita Paribahan,A/C Seater (2+3),23:35,05h 25m,05:00,4.2,INR 370,15 Seats available
3,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,The Joy Ride,A/C Seater / Sleeper (2+2),21:15,06h 20m,03:35,3.5,INR 650,45 Seats available
4,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...,Aradhana Bus Service,A/C Seater / Sleeper (2+2),23:00,05h 30m,04:30,1.9,570,47 Seats available
...,...,...,...,...,...,...,...,...,...,...
63,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Shyamoli Paribahan Pvt Ltd,Volvo 9600 Multi-Axle A/C Sleeper (2+1),20:15,10h 35m,06:50,4.6,INR 1500,10 Seats available
64,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Greenline,Volvo Multi-Axle A/C Sleeper (2+1),22:00,09h 45m,07:45,4.5,INR 1500,6 Seats available
65,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Modern Travels,Bharat Benz A/C Seater /Sleeper (2+1),20:00,11h 45m,07:45,3.6,INR 850,24 Seats available
66,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),20:00,04h 15m,00:15,3.7,523,14 Seats available


SOUTH BENGAL BUS ROUTES

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
            
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
 

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_southbengal = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_southbengal.to_csv('sb_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/burdwan-to-kolkata: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF68CD69632+30946]
	(No symbol) [0x00007FF68CD1E3C9]
	(No symbol) [0x00007FF68CC16FDA]
	(No symbol) [0x00007FF68CC6822C]
	(No symbol) [0x00007FF68CC6850C]
	(No symbol) [0x00007FF68CCADCB7]
	(No symbol) [0x00007FF68CC8CAAF]
	(No symbol) [0x00007FF68CCAB041]
	(No symbol) [0x00007FF68CC8C813]
	(No symbol) [0x00007FF68CC5A6E5]
	(No symbol) [0x00007FF68CC5B021]
	GetHandleVerifier [0x00007FF68CE9F83D+1301229]
	GetHandleVerifier [0x00007FF68CEABDB7+1351783]
	GetHandleVerifier [0x00007FF68CEA2A03+1313971]
	GetHandleVerifier [0x00007FF68CD9DD06+245686]
	(No symbol) [0x00007FF68CD2758F]
	(No symbol) [0x00007FF68CD23804]
	(No symbol) [0x00007FF68CD23992]
	(No symbol) [0x00007FF68CD1A3EF]
	BaseThreadInitThunk [0x00007FFBA00B7614+20]
	RtlUserThreadStart [0x00007FFBA09E26A1+33]

Error occurred while scraping bus details for https://www.redbus.in/bus-tic

NameError: name 'driver' is not defined

In [14]:
df_southbengal

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),20:00,02h 15m,22:15,2.0,380,27 Seats available
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,Suman sabir,Non AC Seater (2+3),21:00,02h 59m,23:59,1.2,INR 111,54 Seats available
2,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,Pammi Travels,A/C Sleeper (2+1),21:00,02h 59m,23:59,3.7,INR 1300,29 Seats available
3,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...,Pammi Travels,Non A/C Seater / Sleeper (2+1),21:00,02h 59m,23:59,3.7,INR 650,33 Seats available
4,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),20:00,03h 15m,23:15,3.7,475,26 Seats available
...,...,...,...,...,...,...,...,...,...,...
864,Purulia to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/purulia-to-d...,SBSTC-BAGMUNDI - KOLKATA - 05:00 (BEL DEPOT) -...,Non AC Seater (2+3),06:50,04h 00m,10:50,3.6,INR 117,52 Seats available
865,Purulia to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/purulia-to-d...,SBSTC-PURULIA - KOLKATA - 07:15 - (VIA ASN) (H...,Non AC Seater (2+3),07:15,03h 50m,11:05,4.0,INR 112,51 Seats available
866,Purulia to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/purulia-to-d...,SBSTC-PURULIA - BERHAMPUR - 07:30 (PRL DEPOT) ...,Non AC Seater (2+3),07:30,03h 50m,11:20,3.4,INR 112,52 Seats available
867,Purulia to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/purulia-to-d...,SBSTC-PURULIA - KOLKATA - VIA - ASN - 09:15 (P...,Non AC Seater (2+3),09:15,04h 15m,13:30,4.0,INR 112,52 Seats available


RAJASTHAN BUS ROUTES

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_rajasthan = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_rajasthan.to_csv('rajasthan_bus_details.csv', index=False)

# Close the driver
driver.quit()

NameError: name 'driver' is not defined

In [16]:
df_rajasthan

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 160781,Super Luxury Volvo AC Seater Pushback 2+2,00:00,14h 00m,14:00,4.6,INR 460,23 Seats available
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 144042,Super Luxury Volvo AC Seater Pushback 2+2,00:00,12h 20m,12:20,4.3,INR 460,37 Seats available
2,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 192566,Semi Deluxe Non AC Seater 2+2,04:45,04h 30m,09:15,4.2,INR 250,35 Seats available
3,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 146314,Super Luxury Volvo AC Seater Pushback 2+2,06:15,04h 00m,10:15,4.3,INR 460,35 Seats available
4,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...,RSRTC - 193343,Semi Deluxe Non AC Seater 2+2,07:15,04h 45m,12:00,4.2,INR 250,35 Seats available
...,...,...,...,...,...,...,...,...,...,...
1245,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,Amar Travels(Jaipur),AC Sleeper (2+1),22:00,04h 45m,02:45,2.4,INR 500,34 Seats available
1246,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,Sharma travels,A/C Seater / Sleeper (2+1),22:30,04h 30m,03:00,1.9,INR 429,34 Seats available
1247,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,Amar Travels(Jaipur),A/C Seater / Sleeper (2+1),22:45,04h 45m,03:30,2.4,INR 400,44 Seats available
1248,Bikaner to Sikar,https://www.redbus.in/bus-tickets/bikaner-to-s...,M R Travels,Volvo Multi-Axle A/C seater/sleeper (2+1),15:15,03h 00m,18:15,3.8,INR 1200,60 Seats available


HIMACHAL BUS ROUTES

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_himachal = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_himachal.to_csv('himachal_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76BE29632+30946]
	(No symbol) [0x00007FF76BDDE3C9]
	(No symbol) [0x00007FF76BCD6FDA]
	(No symbol) [0x00007FF76BD2822C]
	(No symbol) [0x00007FF76BD2850C]
	(No symbol) [0x00007FF76BD6DCB7]
	(No symbol) [0x00007FF76BD4CAAF]
	(No symbol) [0x00007FF76BD6B041]
	(No symbol) [0x00007FF76BD4C813]
	(No symbol) [0x00007FF76BD1A6E5]
	(No symbol) [0x00007FF76BD1B021]
	GetHandleVerifier [0x00007FF76BF5F83D+1301229]
	GetHandleVerifier [0x00007FF76BF6BDB7+1351783]
	GetHandleVerifier [0x00007FF76BF62A03+1313971]
	GetHandleVerifier [0x00007FF76BE5DD06+245686]
	(No symbol) [0x00007FF76BDE758F]
	(No symbol) [0x00007FF76BDE3804]
	(No symbol) [0x00007FF76BDE3992]
	(No symbol) [0x00007FF76BDDA3EF]
	BaseThreadInitThunk [0x00007FFC16467614+20]
	RtlUserThreadStart [0x00007FFC178A26A1+33]

Error occurred while scraping bus details for https://

NameError: name 'driver' is not defined

In [3]:
df_himachal

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 591,A/C Executive (2+3),21:00,11h 15m,08:15,4.0,INR 622,36 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 8,Himsuta AC Seater Volvo/Scania 2+2,21:02,09h 28m,06:30,4.4,INR 892,25 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 570,Ordinary 3+2 Non AC Seater,21:25,09h 20m,06:45,3.6,INR 507,37 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 20,Himsuta AC Seater Volvo/Scania 2+2,21:36,07h 54m,05:30,3.9,INR 867,32 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 518,Ordinary 3+2 Non AC Seater,21:58,09h 17m,07:15,2.5,INR 507,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
458,Kangra to Delhi,https://www.redbus.in/bus-tickets/kangra-to-delhi,Triptara tour & Travel,Scania Multi-Axle AC Semi Sleeper (2+2),21:28,09h 27m,06:55,4.5,INR 500,16 Seats available
459,Kangra to Delhi,https://www.redbus.in/bus-tickets/kangra-to-delhi,Bharmani Travels,Volvo A/C Semi Sleeper (2+2),22:00,08h 30m,06:30,3.9,600,12 Seats available
460,Kangra to Delhi,https://www.redbus.in/bus-tickets/kangra-to-delhi,Northern Travels,Volvo A/C Semi Sleeper (2+2),19:45,09h 30m,05:15,2.5,INR 539,14 Seats available
461,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...,HRTC - 78,Himsuta AC Seater Volvo/Scania 2+2,21:15,07h 45m,05:00,4.5,INR 892,30 Seats available


ASSAM BUS ROUTES

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_assam = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_assam.to_csv('assam_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/guwahati-to-tezpur: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76BE29632+30946]
	(No symbol) [0x00007FF76BDDE3C9]
	(No symbol) [0x00007FF76BCD6FDA]
	(No symbol) [0x00007FF76BD2822C]
	(No symbol) [0x00007FF76BD2850C]
	(No symbol) [0x00007FF76BD6DCB7]
	(No symbol) [0x00007FF76BD4CAAF]
	(No symbol) [0x00007FF76BD6B041]
	(No symbol) [0x00007FF76BD4C813]
	(No symbol) [0x00007FF76BD1A6E5]
	(No symbol) [0x00007FF76BD1B021]
	GetHandleVerifier [0x00007FF76BF5F83D+1301229]
	GetHandleVerifier [0x00007FF76BF6BDB7+1351783]
	GetHandleVerifier [0x00007FF76BF62A03+1313971]
	GetHandleVerifier [0x00007FF76BE5DD06+245686]
	(No symbol) [0x00007FF76BDE758F]
	(No symbol) [0x00007FF76BDE3804]
	(No symbol) [0x00007FF76BDE3992]
	(No symbol) [0x00007FF76BDDA3EF]
	BaseThreadInitThunk [0x00007FFC16467614+20]
	RtlUserThreadStart [0x00007FFC178A26A1+33]

Error occurred while scraping bus details for https://www.redbus.in/bus-tic

NameError: name 'driver' is not defined

In [5]:
df_assam

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,06h 00m,03:30,2.9,INR 400,7 Seats available
1,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,Himalayan Travels,NON AC Seater / Sleeper 2+1,23:40,04h 20m,04:00,1.8,450,17 Seats available
2,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...,DESTINY TRAVELS,NON A/C Seater (2+1),23:45,04h 00m,03:45,1.8,399,2 Seats available
3,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...,Megha Transport,Bharat Benz NON A/C Seater / Sleeper (2+1),19:50,02h 25m,22:15,3.5,500,8 Seats available
4,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...,Yatra,Non AC Seater (2+2),20:15,07h 15m,03:30,2.6,INR 350,40 Seats available
...,...,...,...,...,...,...,...,...,...,...
122,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Aniruddha Travels,AC Seater (2+2),07:30,06h 30m,14:00,3.5,500,44 Seats available
123,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Baikuntha Transport Service,Bharat Benz A/C Sleeper (2+1),19:30,05h 30m,01:00,3.9,500,50 Seats available
124,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Purple Wings Coaches,A/C Seater / Sleeper (2+1),19:55,05h 20m,01:15,3.6,569,43 Seats available
125,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...,Shree Ram Travels,Bharat Benz A/C Seater /Sleeper (2+1),20:45,04h 55m,01:40,4.8,530,42 Seats available


CHANDIGARH BUS ROUTES

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_chandigarh = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_chandigarh.to_csv('chandigarh_bus_details.csv', index=False)

# Close the driver
driver.quit()

NameError: name 'driver' is not defined

In [7]:
df_chandigarh

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165681,HVAC Seater (2+3),07:00,05h 35m,12:35,4.0,INR 404.76,46 Seats available
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165684,HVAC Seater (2+3),07:55,05h 35m,13:30,4.0,INR 404.76,47 Seats available
2,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165687,HVAC Seater (2+3),08:25,05h 35m,14:00,3.8,INR 404.76,47 Seats available
3,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165676,HVAC Seater (2+3),09:25,05h 35m,15:00,4.1,INR 404.76,46 Seats available
4,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165689,HVAC Seater (2+3),10:30,05h 35m,16:05,3.6,INR 404.76,47 Seats available
...,...,...,...,...,...,...,...,...,...,...
1279,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,Zingbus Plus,Volvo Multi-Axle A/C Semisleeper (2+2),02:45,04h 15m,07:00,4.3,676,23 Seats available
1280,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,Asian's Shina Tour & Travels (Swoyambhu Manjus...,A/C Semi Sleeper (2+2),00:29,04h 31m,05:00,2.4,INR 999,1 Seat available
1281,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,Northern Travels,Volvo A/C Semi Sleeper (2+2),01:20,04h 55m,06:15,2.7,INR 600,8 Seats available
1282,Chandigarh to Jawala Ji,https://www.redbus.in/bus-tickets/chandigarh-t...,PAL Travel Lines,Bharat Benz A/C Semi Sleeper (2+2),02:04,04h 01m,06:05,2.9,1619,16 Seats available


JAMMU & KASHMIR BUS ROUTES

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/jksrtc"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df_jammukashmir = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_jammukashmir.to_csv('jk_bus_details.csv', index=False)

# Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-srinagar: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF609669642+30946]
	(No symbol) [0x00007FF60961E3D9]
	(No symbol) [0x00007FF609516FDA]
	(No symbol) [0x00007FF60956822C]
	(No symbol) [0x00007FF60956850C]
	(No symbol) [0x00007FF6095ADCB7]
	(No symbol) [0x00007FF60958CAAF]
	(No symbol) [0x00007FF6095AB041]
	(No symbol) [0x00007FF60958C813]
	(No symbol) [0x00007FF60955A6E5]
	(No symbol) [0x00007FF60955B021]
	GetHandleVerifier [0x00007FF60979F84D+1301229]
	GetHandleVerifier [0x00007FF6097ABDC7+1351783]
	GetHandleVerifier [0x00007FF6097A2A13+1313971]
	GetHandleVerifier [0x00007FF60969DD16+245686]
	(No symbol) [0x00007FF60962759F]
	(No symbol) [0x00007FF609623814]
	(No symbol) [0x00007FF6096239A2]
	(No symbol) [0x00007FF60961A3FF]
	BaseThreadInitThunk [0x00007FFC53377614+20]
	RtlUserThreadStart [0x00007FFC539626A1+33]

Error occurred while scraping bus details for https://www.redbus.in/bus-tick

NameError: name 'driver' is not defined

In [3]:
df_jammukashmir

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),21:15,10h 00m,07:15,1.4,719,34 Seats available
1,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,Harikesh Tour N Travels,NON A/C Seater / Sleeper (2+2),20:30,09h 30m,06:00,1.0,584,44 Seats available
2,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,North Kashmir Tour and Travels,NON A/C Sleeper (2+2),16:30,10h 00m,02:30,1.0,INR 699,40 Seats available
3,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,Jamna Travels-Jammu,Non AC Seater (2+2),21:30,10h 30m,08:00,2.4,INR 549,29 Seats available
4,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,North Kashmir Tour and Travels,NON A/C Sleeper (2+2),17:15,10h 00m,03:15,1.0,INR 699,40 Seats available
...,...,...,...,...,...,...,...,...,...,...
205,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Royal Golden,A/C Seater / Sleeper (2+2),14:00,750h 00m,20:00,1.8,600,35 Seats available
206,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,AC Sleeper (2+1),14:30,05h 30m,20:00,2.2,740,23 Seats available
207,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Neeraj travels,A/C Seater / Sleeper (2+1),15:40,04h 32m,20:12,2.5,759,38 Seats available
208,Jammu (j and k) to Amritsar,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Sleeper (2+1),17:15,04h 35m,21:50,2.2,694,27 Seats available


KADAMBA BUS ROUTES

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                     "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 5):  # There are 4 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()


# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kadamba_bus_details.csv', index=False)

# Close the driver
driver.quit()




Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/calangute-goa-to-mopa-airport: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF70FAD9632+30946]
	(No symbol) [0x00007FF70FA8E3C9]
	(No symbol) [0x00007FF70F986FDA]
	(No symbol) [0x00007FF70F9D822C]
	(No symbol) [0x00007FF70F9D850C]
	(No symbol) [0x00007FF70FA1DCB7]
	(No symbol) [0x00007FF70F9FCAAF]
	(No symbol) [0x00007FF70FA1B041]
	(No symbol) [0x00007FF70F9FC813]
	(No symbol) [0x00007FF70F9CA6E5]
	(No symbol) [0x00007FF70F9CB021]
	GetHandleVerifier [0x00007FF70FC0F83D+1301229]
	GetHandleVerifier [0x00007FF70FC1BDB7+1351783]
	GetHandleVerifier [0x00007FF70FC12A03+1313971]
	GetHandleVerifier [0x00007FF70FB0DD06+245686]
	(No symbol) [0x00007FF70FA9758F]
	(No symbol) [0x00007FF70FA93804]
	(No symbol) [0x00007FF70FA93992]
	(No symbol) [0x00007FF70FA8A3EF]
	BaseThreadInitThunk [0x00007FF8951C7614+20]
	RtlUserThreadStart [0x00007FF895AA26A1+33]

Error occurred while scraping bus details for https://www.redbus

NameError: name 'driver' is not defined

In [2]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,10h 35m,05:50,3.6,INR 900,15 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.5,INR 555,19 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:00,11h 30m,08:30,4.5,INR 660,19 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Amrutyog,A/C Sleeper (2+1),22:00,11h 30m,09:30,4.4,INR 660,25 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Ashray Travels,Non A/C Seater / Sleeper (2+1),21:15,10h 30m,07:45,4.5,INR 400,30 Seats available
...,...,...,...,...,...,...,...,...,...,...
350,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,00h 31m,16:01,3.7,INR 100,41 Seats available
351,Calangute (goa) to Panaji,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,Volvo AC Seater 2+2,13:30,00h 30m,14:00,3.7,INR 100,39 Seats available
352,Calangute (goa) to Panaji,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,00h 30m,16:00,3.7,INR 100,41 Seats available
353,Goa Airport to Panaji,https://www.redbus.in/bus-tickets/goa-airport-...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),13:45,01h 00m,14:45,4.4,INR 150,41 Seats available
